<a href="https://colab.research.google.com/github/waheed444/AgenticAI-Playground/blob/main/Project02_Langchain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set-up the Environment & Install Prerequisites

In [1]:
! pip install -q --upgrade google-generativeai langchain langchain-community langchain-google-genai chromadb pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.5 MB/s eta 0:00:

### Display the output in  Prope Formate

In [15]:
from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Import **genai** & Configure **API Key**

In [2]:
import os
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

### Import **Libraries** to build QnA using **LangChain and PDF Data** Integration

In [3]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

warnings.filterwarnings("ignore")
# restart python kernal if issues with langchain import.

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

### Configure **gimini-pro** model with custom parameters

In [5]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,temperature=0.6,convert_system_message_to_human=True)

### Checking file Existence using **pathlib** library

In [8]:
from pathlib import Path

file_path = Path("/content/RAG_File.pdf")
if file_path.exists():
    print("File exists and is ready for processing.")
else:
    print("File does not exist. Please upload it.")


File exists and is ready for processing.


### Loading & Splitting PDF Content Using **PyPDFLoader** library

In [9]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("/content/RAG_File.pdf")
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

technologies.Useopensourcelibraries,likeLangchain,CrewAI,andLangGraphtoautomaterepeatable,multi-steptasksandautomatebusinessprocessesthataretypicallydonebyagroupofpeople.
Certifications:
■ MicrosoftCertified:AzureAIEngineerAssociate■ CertifiedcrewAIEngineer
LearningRepo:https://github.com/panaversity/learn-prompt-eng-gpts-ai-agents
● Quarter3:CloudNativeAIPoweredMicroservicesDesign,Development,andDeployment:
BuildscalableAIPoweredAPIsusingFastAPI,Postgres,Kafka,Kong,GenAIAPIslikeOpenAIChatCompletionAPIs,AssistantAPIs,LangChainandOpenSourceAILLMs,developthemusingContainersandDevContainers,anddeploythemusingDockerComposelocallyandKubernetesPoweredServerlessContainerServicesonthecloud.
WewillalsolearntointegratedesignthinkingandBehavior-DrivenDevelopment(BDD)indevelopingAIsystems.WewilllearntocreateAIsolutionsthataredeeplyalignedwithuserneedsandexpectations.Designthinkingensuresathoroughunderstandingoftheuserandproblemspace,whileBDDprovidesastructuredapproachtodefiningandvalidatingthedesi

### Splitting Text into Chunks for Embedding Using **LangChain and Google Generative AI**

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

In [12]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

### Creating a Retrieval-Based QnA System with **Google Generative AI Embeddings and Chroma**

In [13]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True

)

### Querying and Displaying Output

In [18]:
question = "What are the main topics discussed in this file?"
result = qa_chain({"query": question})
result["result"]
Markdown(result["result"])

This file discusses the following topics:
1. Autonomy of AIAgents
2. Learning and Adaptation of AIAgents
3. Interactivity of AIAgents
4. Design Thinking and Behavior-Driven Development for AIAgents and Custom GPTs
5. Fine-Tuning Open-Source Large Language Models with FastAI and PyTorch
6. Importance of PyTorch and FastAI in Fine-Tuning LLMs
7. Why TensorFlow is not preferred over PyTorch and FastAI for fine-tuning LLMs
8. Challenges in Developing LLMs like ChatGPT4 or Google Gemini
9. Business Considerations for Developing Custom LLMs vs Using Existing LLMs
10. What are Custom GPTs?
11. What are Actions in GPTs?
12. Differences between AIAgents and Custom GPTs
13. Data Preparation for Fine-Tuning MetaLLaMA3 with PyTorch
14. Fine-Tuning MetaLLaMA3 with PyTorch
15. Utilizing FastAI for NLP Tasks
16. Cloud-Native Training and Deployment for Fine-Tuned LLMs
17. Exporting Models for Inference and Building Inference Pipelines
18. Capstone Project for Fine-Tuning and Deploying MetaLLaMA3
19. Physical AI and Humanoid Robotics Development
20. Cloud Native Microservices Deployment and Distributed System Design
21. Front-end Web GUI Development using Next.js and TypeScript
22. What is Cloud Native Applied Generative AI Engineering?
23. What is Physical AI?
24. Different Specializations offered at the end of the program and their benefits
25. Purpose of Docker Containers and benefits of deploying them with Docker Compose and Kubernetes
26. Purpose of learning to develop APIs in a Generative AI program
27. Purpose of using Python-based FastAPI and related technologies in Quarter 3
28. API-as-a-Product model
29. Benefits of using Docker Containers for development, testing, and deployment
30. Advantages of using open Docker, Kubernetes, and Terraform technologies instead of using AWS, Azure, or Google Cloud technologies
31. Reasons for not learning to build LLMs in the program and the difficulty of developing LLMs like ChatGPT4 or Google Gemini